In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch

In [2]:
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'

In [3]:
data = pd.read_csv('../input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict,
                       index_col = 0, chunksize = 3141410) #3141410

In [4]:
data = next(iter(data))

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
target = data['target']
data.drop(['target', 'investment_id'], inplace=True, axis=1)

In [6]:
#data.groupby(['time_id'])['target'].std()

In [7]:
"""
from scipy.stats import pearsonr 
def est_score(model):
    y_pred = model.predict(X_test)
    return pearsonr(y_pred, y_test)[0]
"""

'\nfrom scipy.stats import pearsonr \ndef est_score(model):\n    y_pred = model.predict(X_test)\n    return pearsonr(y_pred, y_test)[0]\n'

In [8]:
#from sklearn.model_selection import TimeSeriesSplit


In [9]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.08, stratify=data.index)
"""

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.08, stratify=data.index)\n'

In [10]:
"""
class DNN_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = torch.nn.Linear(300, 180)
        self.linear_2 = torch.nn.Linear(180, 180)
        self.linear_3 = torch.nn.Linear(180, 1)
        self.act = torch.nn.SiLU()
    def forward(self, X):
        X = self.act(self.linear_1(X))
        X = self.act(self.linear_2(X))
        X = self.linear_3(X)
        return X
"""

'\nclass DNN_Model(torch.nn.Module):\n    def __init__(self):\n        super().__init__()\n        self.linear_1 = torch.nn.Linear(300, 180)\n        self.linear_2 = torch.nn.Linear(180, 180)\n        self.linear_3 = torch.nn.Linear(180, 1)\n        self.act = torch.nn.SiLU()\n    def forward(self, X):\n        X = self.act(self.linear_1(X))\n        X = self.act(self.linear_2(X))\n        X = self.linear_3(X)\n        return X\n'

In [11]:
"""
tensor_train = torch.FloatTensor(X_train.values)
tensor_test = torch.FloatTensor(X_test.values)
print(tensor_train.shape)
tensor_y = torch.FloatTensor(y_train.values)
tensor_y_test = (y_test.values)

model = DNN_Model()

opt = torch.optim.Adam(model.parameters(), lr=5e-4)
for epoch in range(100):
    model.zero_grad()
    loss = torch.nn.MSELoss()(model(tensor_train).view(-1), tensor_y)
    loss.backward()
    opt.step()
    print(f'epoch {epoch} finished with {loss} loss')
    y_pred = model(tensor_test).view(-1)
    print(pearsonr(y_pred.detach().numpy(), tensor_y_test)[0])
"""

"\ntensor_train = torch.FloatTensor(X_train.values)\ntensor_test = torch.FloatTensor(X_test.values)\nprint(tensor_train.shape)\ntensor_y = torch.FloatTensor(y_train.values)\ntensor_y_test = (y_test.values)\n\nmodel = DNN_Model()\n\nopt = torch.optim.Adam(model.parameters(), lr=5e-4)\nfor epoch in range(100):\n    model.zero_grad()\n    loss = torch.nn.MSELoss()(model(tensor_train).view(-1), tensor_y)\n    loss.backward()\n    opt.step()\n    print(f'epoch {epoch} finished with {loss} loss')\n    y_pred = model(tensor_test).view(-1)\n    print(pearsonr(y_pred.detach().numpy(), tensor_y_test)[0])\n"

In [12]:
"""
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression(copy_X=False).fit(X_train, y_train)
"""

'\nfrom sklearn.linear_model import LinearRegression\nreg_model = LinearRegression(copy_X=False).fit(X_train, y_train)\n'

In [13]:
#est_score(reg_model)

In [14]:
"""
from catboost import CatBoostRegressor

class BoostStack:
    def __init__(self, n_est):
        self.boosts = [CatBoostRegressor(iterations=100, subsample=0.7, silent = True) for i in range(n_est)]
    
    def fit(self, X, y):
        for i in range(len(self.boosts)):
            self.boosts[i].fit(X, y)
            print(f'{i}-th boost fitted')
    
    def predict(self, X):
        res = 0
        
        for i in range(len(self.boosts)):
            res += self.boosts[i].predict(X) 
        return res / len(self.boosts)
    
"""

"\nfrom catboost import CatBoostRegressor\n\nclass BoostStack:\n    def __init__(self, n_est):\n        self.boosts = [CatBoostRegressor(iterations=100, subsample=0.7, silent = True) for i in range(n_est)]\n    \n    def fit(self, X, y):\n        for i in range(len(self.boosts)):\n            self.boosts[i].fit(X, y)\n            print(f'{i}-th boost fitted')\n    \n    def predict(self, X):\n        res = 0\n        \n        for i in range(len(self.boosts)):\n            res += self.boosts[i].predict(X) \n        return res / len(self.boosts)\n    \n"

In [15]:
#est_score(model)

In [16]:
"""
scores = []
iters = [10, 20, 30, 50, 100, 200, 500, 1000, 2000, 5000]
for it in iters:
    md = CatBoostRegressor(iterations=it, task_type='GPU', devices='0', silent=True).fit(X_train, y_train)
    scores.append(est_score(md))
    print(f'finished with {it}')
    print(scores[-1])
"""

"\nscores = []\niters = [10, 20, 30, 50, 100, 200, 500, 1000, 2000, 5000]\nfor it in iters:\n    md = CatBoostRegressor(iterations=it, task_type='GPU', devices='0', silent=True).fit(X_train, y_train)\n    scores.append(est_score(md))\n    print(f'finished with {it}')\n    print(scores[-1])\n"

In [17]:
#sns.lineplot(x=iters, y=scores)

from sklearn.linear_model import LinearRegression
model = LinearRegression(copy_X=False).fit(data.drop('investment_id', axis=1), target)

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_to(data, device):
    """
    moving data to device
    :param data: data to move
    :param device: device
    :return: moved data
    """
    if isinstance(data, (list, tuple)):
        return [move_to(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [19]:
class DNN_Model(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear_3 = torch.nn.Linear(deep_size, deep_size)
        self.linear_4 = torch.nn.Linear(deep_size, deep_size)
        self.linear_5 = torch.nn.Linear(deep_size, 1)
        self.dout = torch.nn.Dropout(p=0.7)
        self.act = torch.nn.SiLU()
    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        self = move_to(self, device)
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]:             
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[i:i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[i:i+batch_size].values), device)
                loss = torch.nn.MSELoss()(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                val_loss = torch.nn.MSELoss()(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.linear_5(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        tensor_X = move_to(tensor_X, device)
        tensor_X = self.act(self.linear_1(tensor_X))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.linear_5(tensor_X)
        return tensor_X

In [20]:
#написать subsample: генерируем подмножество всех индексов и из него выбираем в батчах
#то есть двойная индексация

In [21]:
class StackedDNN:
    def __init__(self, n, min_deep=30, max_deep=40, min_iter=10, max_iter=15):
        self.nets = [DNN_Model(np.random.randint(min_deep, max_deep)) for i in range(n)]
        self.min_iter=min_iter
        self.max_iter=max_iter
    
    def fit(self, X, y, batch_size=4096, subset=0.8):
        for i in range(len(self.nets)):
            self.nets[i].fit(X, y, np.random.randint(self.min_iter, self.max_iter), batch_size, subset, silent=True, plot=False, val=False, X_val=None, y_val=None)
            print(f'{i}/{len(self.nets)-1} fitted')
        
    def predict(self, X):
        y_pred = np.ndarray([len(self.nets), X.shape[0]])
        for i in range(len(self.nets)):
            self.nets[i].eval()
            y_pred[i] = np.array(self.nets[i].forward(X).detach().view(-1).cpu())
        y_pred /= len(self.nets)
        return np.sum(y_pred, axis=0)

In [22]:
nn_model = StackedDNN(20, min_iter=5,max_iter=7, min_deep=280, max_deep=301)

In [23]:
nn_model.fit(data, target)

0/19 fitted
1/19 fitted
2/19 fitted
3/19 fitted
4/19 fitted
5/19 fitted
6/19 fitted
7/19 fitted
8/19 fitted
9/19 fitted
10/19 fitted
11/19 fitted
12/19 fitted
13/19 fitted
14/19 fitted
15/19 fitted
16/19 fitted
17/19 fitted
18/19 fitted
19/19 fitted


In [24]:
#res = nn_model.predict(X_test)

In [25]:
#pearsonr(res, y_test)[0]

In [26]:
#np.sum(res > 10000000000000)

In [27]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=8000, task_type='GPU', devices='0', silent=True).fit(data, target)

In [28]:
#est_score(model)

In [29]:
#res_boost = model.predict(X_test)

In [30]:
#a = 0.5
#pearsonr(a*res_boost+(1-a)*res, y_test)[0]

In [31]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    res_boost = model.predict(test_df[features])
    res_nn = nn_model.predict(test_df[features])
    test_df['target']  = 0.5*res_boost + 0.5*res_nn
    env.predict(test_df[['row_id','target']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [32]:
.902

0.902